<a href="https://colab.research.google.com/github/ShinjiNeo/Test_ML/blob/main/Template_Regressione_Pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIPELINE Regressione


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- importo librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer,OneHotEncoder,StandardScaler,FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

- importo il dataset

In [ ]:
# Creazione del dataset
np.random.seed(42)

# Numero di righe del dataset
n = 500

# Generazione delle colonne
data = {
    'ID': range(1, n+1),
    'Superficie': np.random.randint(50, 200, size=n),  # Superficie tra 50 e 200 m²
    'Numero di stanze': np.random.randint(2, 6, size=n),  # Stanze tra 2 e 5
    'Età della casa': np.random.randint(1, 50, size=n),  # Età tra 1 e 50 anni
    'Distanza dal centro': np.random.randint(1, 20, size=n),  # Distanza tra 1 e 20 km
    'Tipo di casa': np.random.choice(['Appartamento', 'Villa', 'Casa a schiera'], size=n),
    'Piscina': np.random.choice([0, 1], size=n),  # 0: no piscina, 1: piscina
    'Prezzo': np.random.randint(100000, 500000, size=n)  # Prezzo tra 100k e 500k
}

# Creazione del DataFrame
df = pd.DataFrame(data)

# Introduzione di valori mancanti
df.loc[df.sample(frac=0.1).index, 'Superficie'] = np.nan
df.loc[df.sample(frac=0.1).index, 'Numero di stanze'] = np.nan
df.loc[df.sample(frac=0.1).index, 'Distanza dal centro'] = np.nan
df.loc[df.sample(frac=0.1).index, 'Prezzo'] = np.nan


In [ ]:
path =''
df = pd.read_csv(path,) # Attenzione ai separatori che possono essere: sep=',' sep=';' sep='\t')
df

- l'analisi l'ho già fatta quindi rimuovo direttamente le feature che non mi interessano

In [ ]:
df1=df.copy()

In [ ]:
features_to_remove = ['ID',]
df_clean = df1.drop(features_to_remove, axis=1)
df_clean

,Superficie,Numero di stanze,Età della casa,Distanza dal centro,Tipo di casa,Piscina,Prezzo
0,152.0,5.0,20,NaN,Villa,0,215359.0
1,142.0,5.0,33,7.0,Villa,0,166707.0
2,64.0,2.0,20,14.0,Casa a schiera,1,351351.0
3,156.0,2.0,49,15.0,Villa,0,338511.0
4,121.0,3.0,12,NaN,Appartamento,1,335718.0
...,...,...,...,...,...,...,...
495,76.0,3.0,47,18.0,Villa,0,250158.0
496,171.0,3.0,32,12.0,Villa,0,423494.0
497,54.0,3.0,10,6.0,Villa,1,433569.0
498,78.0,5.0,16,13.0,Villa,0,448264.0


In [ ]:
X = df_clean.drop(['Prezzo'], axis=1) # matrice Features
y = df_clean['Prezzo'].fillna(df_clean['Prezzo'].median())

***Attenzione successivamente non verranno fillati i valori nulli nella y perciò magari farlo qui***

In [ ]:
X.columns

Index(['Superficie', 'Numero di stanze', 'Età della casa',
       'Distanza dal centro', 'Tipo di casa', 'Piscina'],
      dtype='object')

- separo le features categoriche da quelle numeriche

In [ ]:
numerical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind in ['i','f'] ]
categorical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind not in ['i','f']]

print(numerical_features)
print(categorical_features)

['Superficie', 'Numero di stanze', 'Età della casa', 'Distanza dal centro', 'Piscina']
['Tipo di casa']


In [ ]:
Preprocessor = make_column_transformer(
                                        (make_pipeline(
                                        SimpleImputer(strategy = 'median'),
                                        KBinsDiscretizer(n_bins=3)), numerical_features), # devo vedere a che serve

                                        (make_pipeline(
                                        #FunctionTransformer(lambda x: np.where(x == 'n.d.', np.nan, x)),
                                        SimpleImputer(strategy = 'constant', fill_value = 'missing'),
                                        OneHotEncoder(categories = 'auto', handle_unknown = 'ignore',drop='first')), categorical_features,
                                        ))

- Parametri con il best fit

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

classifier = GradientBoostingRegressor(learning_rate= 0.01,
                                    max_depth= 5,
                                    min_samples_split= 5,
                                    n_estimators= 50
                                       )

- Ecco realizzata la PIPE

In [ ]:
model_pipe = Pipeline([
                       ('Preprocessing features', Preprocessor),
                       ('Scaling and standadize data', scaler),
                       ('Classifier', classifier)
                      ])
model_pipe

Pipeline(steps=[('Preprocessing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['Superficie',
                                                   'Numero di stanze',
                                                   'Età della casa',
                                                   'Distanza dal centro',
                                                   'Piscina']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Tipo di casa'])])),
                ('Scaling and standadize data', StandardScaler()),
                ('Classifier',
                 GradientBoostingRegressor(learning_rate=0.01, max_depth=5,
                                           min_samples_split=5,
                                           n_estimators=50))])

- Qui abbiamo la cross validation

In [ ]:
cross_val_scores = cross_val_score(model_pipe, X, y, cv=5)
print(cross_val_scores)

# Calcola la media e la deviazione standard dei punteggi
mean_score = np.mean(cross_val_scores)
std_score = np.std(cross_val_scores)

print(f"Mean cross-validation score:{mean_score:.2f}")
print(f"Standard deviation of cross-validation scores:{std_score:.2f}")

[-0.0855883  -0.02553622 -0.06590679  0.01088745 -0.02241822]
Mean cross-validation score:-0.04
Standard deviation of cross-validation scores:0.03


## FIt del modello con la pipe la quale fa:

- operazioni di preprocessing


In [ ]:
model_pipe.fit(X, y)

Pipeline(steps=[('Preprocessing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['Superficie',
                                                   'Numero di stanze',
                                                   'Età della casa',
                                                   'Distanza dal centro',
                                                   'Piscina']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Tipo di casa'])])),
                ('Scaling and standadize data', StandardScaler()),
                ('Classifier',
                 GradientBoostingRegressor(learning_rate=0.01, max_depth=5,
                                           min_samples_split=5,
                                           n_estimators=50))])

In [ ]:
y_pred_tot = model_pipe.predict(X)
y_pred_tot

array([289149.51873071, 296931.59801178, 309322.74963997, 278398.08864792,
       298832.48282455, 289596.75094098, 271939.54419071, 289596.75094098,
       264331.65973571, 284300.64626617, 299912.25051275, 283903.88711259,
       307700.54014708, 312226.66237766, 289596.75094098, 298534.08533213,
       251806.83543914, 292546.40029792, 298534.08533213, 312226.66237766,
       278398.08864792, 300718.51686332, 278398.08864792, 298832.48282455,
       312425.25196329, 290701.23611654, 293212.67974553, 300527.90415078,
       301987.01169028, 289418.90075078, 300675.40802465, 313236.98949592,
       278894.65044271, 309394.01340336, 312425.25196329, 296931.59801178,
       289596.75094098, 307228.99801697, 296931.59801178, 317714.20469425,
       289149.51873071, 315051.02233856, 280368.18957812, 271939.54419071,
       313236.98949592, 300675.40802465, 284300.64626617, 313236.98949592,
       289596.75094098, 307228.99801697, 288366.9861066 , 289596.75094098,
       293414.9054415 , 2

## Predizione

In [ ]:
X.columns

Index(['Superficie', 'Numero di stanze', 'Età della casa',
       'Distanza dal centro', 'Tipo di casa', 'Piscina'],
      dtype='object')

In [ ]:
df_clean.describe().T

,count,mean,std,min,25%,50%,75%,max
Superficie,450.0,126.408889,43.794844,50.0,88.00,130.0,164.00,199.0
Numero di stanze,450.0,3.437778,1.109543,2.0,2.00,3.0,4.00,5.0
Età della casa,500.0,25.348000,14.036557,1.0,12.75,26.0,37.00,49.0
Distanza dal centro,450.0,10.017778,5.371263,1.0,5.00,10.0,14.75,19.0
Piscina,500.0,0.524000,0.499924,0.0,0.00,1.0,1.00,1.0
Prezzo,450.0,295401.108889,113081.864162,100781.0,196200.25,299028.0,388920.50,499932.0


In [ ]:
Superficie = 200
Numero_di_stanze = 5
Eta_della_casa = 2
Distanza_dal_centro = 1
Tipo_di_casa = 0
Piscina = 1


data = pd.DataFrame({
    "Superficie": [Superficie],
    "Numero di stanze": [Numero_di_stanze],
    "Età della casa": [Eta_della_casa],
    "Distanza dal centro": [Distanza_dal_centro],
    "Tipo di casa": [Tipo_di_casa],
    "Piscina": [Piscina]
})

prezzo_predetto = model_pipe.predict(data)

print(f"Prezzo previsto: {prezzo_predetto[0]:,.2f} €")

Prezzo previsto: 301,691.75 €


## Esportazione del modello in PLK

In [ ]:
import joblib

joblib.dump(model_pipe,'Price_house_pred.pkl')

['Price_house_pred.pkl']

In [ ]:
Price_house_pred=joblib.load('Price_house_pred.pkl')
Price_house_pred

Pipeline(steps=[('Preprocessing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['Superficie',
                                                   'Numero di stanze',
                                                   'Età della casa',
                                                   'Distanza dal centro',
                                                   'Piscina']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Tipo di casa'])])),
                ('Scaling and standadize data', StandardScaler()),
                ('Classifier',
                 GradientBoostingRegressor(learning_rate=0.01, max_depth=5,
                                           min_samples_split=5,
                                           n_estimators=50))])

In [ ]:
prezzo_predetto = Price_house_pred.predict(data)
print(f"Prezzo previsto: {prezzo_predetto[0]:,.2f} €")


Prezzo previsto: 301,691.75 €
